In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import pickle
import numpy as np
from mayavi import mlab
from plyfile import PlyData
import networkx as nx
from scipy import sparse, spatial

In [2]:
def load_skels_from_dir(directory):
    pickle_path = os.path.join(directory, "parsed_graph.pickle")

    if os.path.exists(pickle_path):
        with open(pickle_path, "rb") as fi:
            print("Loading skeleton data from pickle")
            vertices, edges, vertex_properties, G = pickle.load(fi)
            return [vertices, edges, vertex_properties, G]
    else:
        print("Loading skeleton data from directory")
        files = [os.path.join(directory, fi)
                 for fi in os.listdir(directory) if fi.endswith(".ply")]
        skel1 = PlyData.read(files[0])
        vertices = np.array(skel1["vertex"].data.tolist())
        vertex_properties = vertices[:, [0, 1]]
        vertices = vertices[:, 2:]
        edges = np.array(skel1["edge"].data.tolist())
        begin_verts = vertices[edges[:, 0], :]
        end_verts = vertices[edges[:, 1], :]
        vector_components = end_verts - begin_verts

        # Load a list of ply files
        for fi in files[1:]:
            skel = PlyData.read(fi)
            _vertices = np.array(skel["vertex"].data.tolist())
            _vertex_properties = _vertices[:, [0, 1]]
            _vertices = _vertices[:, 2:]
            _edges = np.array(skel["edge"].data.tolist())

            # Increment all the indices in edges so that they match up with the full vertex array
            edges = np.concatenate([edges, (_edges + vertices.shape[0])], axis=0)

            vertices = np.concatenate([vertices, _vertices], axis=0)
            vertex_properties = np.concatenate([vertex_properties, _vertex_properties], axis=0)

        G = nx.Graph()
        G.add_edges_from(edges)

        with open(pickle_path, "wb") as fi:
            print("Writing skeleton data as pickle")
            pickle.dump([vertices, edges, vertex_properties, G], fi)

    return [vertices, edges, vertex_properties, G]


def make_filtered_graph(_vertices, _vertex_properties, _edges, threshold):
    vertex_filter = _vertex_properties[:, 1] > threshold
    good_indices = np.arange(_vertices.shape[0])[vertex_filter]
    edge_filter = np.logical_and(np.isin(_edges[:, 0], good_indices),
                                 np.isin(_edges[:, 1], good_indices))
    filtered_verts = _vertices[vertex_filter, :]
    filtered_edges = _edges[edge_filter, :]
    filtered_begin_verts = _vertices[filtered_edges[:, 0], :]
    filtered_end_verts = _vertices[filtered_edges[:, 1], :]
    filtered_vector_components = filtered_end_verts - filtered_begin_verts
    filtered_distances = np.sqrt(np.sum(np.square(filtered_end_verts - filtered_begin_verts), axis=1))
    mlab.points3d(filtered_verts[:, 0], filtered_verts[:, 1], filtered_verts[:, 2],
                  np.sqrt(vertex_properties[vertex_filter, 1]),
                  opacity=0.1)

    mlab.quiver3d(filtered_begin_verts[:, 0], filtered_begin_verts[:, 1], filtered_begin_verts[:, 2],
                  filtered_vector_components[:, 0], filtered_vector_components[:, 1],
                  filtered_vector_components[:, 2], scalars=filtered_distances, scale_mode="scalar", scale_factor=1,
                  mode="2ddash", opacity=1.)


def plot_subgraph_old(vertices, edges, vertex_properties, subgraph_no):
    _, subgraph_edges = get_subgraph(edges, subgraph_no)
    begin_verts = vertices[subgraph_edges[:, 0], :]
    end_verts = vertices[subgraph_edges[:, 1], :]
    vector_components = end_verts - begin_verts
    distances = np.sqrt(np.sum(np.square(end_verts - begin_verts), axis=1))
    vertex_mask = np.isin(np.arange(vertices.shape[0]), subgraph_edges)

    mlab.points3d(vertices[vertex_mask, 0], vertices[vertex_mask, 1], vertices[vertex_mask, 2],
                  vertex_properties[vertex_mask, 1], scale_mode='scalar', scale_factor=1, opacity=0.1)
    mlab.quiver3d(begin_verts[:, 0], begin_verts[:, 1], begin_verts[:, 2], vector_components[:, 0],
                  vector_components[:, 1],
                  vector_components[:, 2], scalars=distances, mode="2ddash", opacity=1., scale_mode="scalar",
                  scale_factor=1, color=(1, 1, 1))

def plot_subgraph(subgraph, vertices, points_kwargs={}, quiver_kwargs={}):
    subgraph_edges = np.array(subgraph.edges)
    begin_verts = vertices[subgraph_edges[:, 0], :]
    end_verts = vertices[subgraph_edges[:, 1], :]
    vector_components = end_verts - begin_verts
    distances = np.sqrt(np.sum(np.square(end_verts - begin_verts), axis=1))
    vertex_mask = np.unique(subgraph_edges.reshape((-1, 1)))

    mlab.points3d(vertices[vertex_mask, 0], vertices[vertex_mask, 1], vertices[vertex_mask, 2],
                  scale_mode='scalar', scale_factor=1, opacity=0.1, **points_kwargs)
    mlab.quiver3d(begin_verts[:, 0], begin_verts[:, 1], begin_verts[:, 2], vector_components[:, 0],
                  vector_components[:, 1],
                  vector_components[:, 2], scalars=distances, mode="2ddash", opacity=1., scale_mode="scalar",
                  scale_factor=1, **quiver_kwargs)

def get_subgraph(edges, subgraph_no):
    G = nx.Graph()
    G.add_edges_from(edges)
    subgraphs = list(sorted(nx.connected_components(G), key=len, reverse=True))
    root_subgraph = G.subgraph(subgraphs[subgraph_no])
    subgraph_edges = np.array(root_subgraph.edges)
    return root_subgraph, subgraph_edges


def plot_skeleton(skel):
    vertices, edges, vertex_properties = get_skel_data(skel)

    G = nx.Graph()
    G.add_edges_from(edges)

    plot_edges(edges, vertices)

    return plot_nodes(vertices, vertex_properties)


def plot_nodes(vertices, vertex_properties, **kwargs):
    plot_args = {"opacity": 0.05, "scale_mode": 'scalar', "scale_factor": 1, }
    plot_args.update(kwargs)
    return mlab.points3d(vertices[:, 0], vertices[:, 1], vertices[:, 2], vertex_properties[:, 1], **kwargs)


def get_skel_data(skel):
    vertices = np.array(skel["vertex"].data.tolist())
    vertex_properties = vertices[:, [0, 1]]
    vertices = vertices[:, 2:]
    edges = np.array(skel["edge"].data.tolist())
    return vertices, edges, vertex_properties


def plot_edges(edges, vertices, **kwargs):
    plot_args = {"mode": "2ddash", "opacity": 1., "scale_mode": "scalar",
                  "scale_factor": 1.}
    plot_args.update(kwargs)
    begin_verts = vertices[edges[:, 0], :]
    end_verts = vertices[edges[:, 1], :]
    vector_components = end_verts - begin_verts
    distances = np.sqrt(np.sum(np.square(end_verts - begin_verts), axis=1))
    return mlab.quiver3d(begin_verts[:, 0], begin_verts[:, 1], begin_verts[:, 2], vector_components[:, 0],
                         vector_components[:, 1],
                         vector_components[:, 2], scalars=distances, **plot_args)

# plot_subgraph(vertices, edges, vertex_properties, 0)

# skel1 = PlyData.read("v2highres_skel.ply")
# vertices, edges, vertex_properties = get_skel_data(skel1)
# # plot_skeleton(skel1)
# plot_edges(edges, vertices)
# plot_edges(cycle1, vertices, color=(1, 0, 0), line_width=10.0)

def get_adjacent_points(center, vertices, side_length):
    half = side_length/2.
    cx, cy, cz = center
    vx = vertices[:, 0]
    vy = vertices[:, 1]
    vz = vertices[:, 2]
    selected = vertices[np.logical_and.reduce(
        (vx >= (cx-half), vx <= (cx+half),
         vy >= (cy-half), vy <= (cy+half),
         vz >= (cz-half), vz <= (cz+half))), :]
    return selected

def get_subgraph_vector(subgraph, vertices, mean_root_vert, get_distal_leaf=False):
    degrees = np.array(subgraph.degree())
    leaf_indices = degrees[degrees[:, 1] == np.min(degrees[:, 1]), 0]
    leaf_verts = vertices[leaf_indices, :]
#     try:
    distal_leaf = leaf_verts[np.argmax(spatial.distance.cdist(mean_root_vert, leaf_verts)), :]
#     except Exception as e:
#         print("Degrees: {}".format(degrees))
#         print("Leaf verts:\n{}".format(leaf_verts))
#         print("Leaf indices:\n{}".format(leaf_indices))
#         raise e
    root_vector = np.mean(vertices[subgraph.nodes, :] - distal_leaf, axis=0)
    norm = np.linalg.norm(root_vector)
    if norm != 0:
        root_vector = root_vector/norm
    if not get_distal_leaf:
        return root_vector.reshape([1, -1])
    else:
        return root_vector.reshape([1, -1]), distal_leaf.reshape([1, -1])
    
def get_elliptical_distance(s_verts, d_verts, alpha, s_vectors, beta, d_vectors):
    # s_verts: m x 3
    # d_verts: o x 3
    # f(a,b): o x 1 x m x 3
    d_verts_br = d_verts[:, None, None, :]
    
    dxdxdz = d_verts_br-s_verts
    
#     vectors = dxdxdz[0,0,:,:]
#     norms = np.linalg.norm(vectors, axis=1)
#     print(s_verts)
#     print(vectors)
#     print(norms)
    
#     dest_vertex = d_verts[0, :].reshape([1, -1])
#     mlab.points3d(dest_vertex[:, 0], dest_vertex[:, 1], dest_vertex[:,2], color=)
    
#     mlab.quiver3d(s_verts[:, 0], s_verts[:, 1], s_verts[:, 2], vectors[:, 0],
#                      vectors[:, 1],
#                      vectors[:, 2], scalars=norms, scale_mode="scalar", scale_factor=1, line_width=5)
    
    dx = dxdxdz[:, :, :, 0]
    dy = dxdxdz[:, :, :, 1]
    dz = dxdxdz[:, :, :, 2]
    sq_distance = dx*dx + dy*dy + dz*dz
#     alpha = 0.9
#     beta = 0.9
    final_distance = np.sqrt(sq_distance - 
                      alpha * np.square((dx*s_vectors[:, 0]+dy*s_vectors[:, 1]+dz*s_vectors[:, 2])) -
                      beta  * np.square((dx*d_vectors[:, 0]+dy*d_vectors[:, 1]+dz*d_vectors[:, 2])))
    final_distance = np.transpose(final_distance.reshape(d_verts.shape[0], s_verts.shape[0]))
    return(final_distance)

# SUBGRAPH_SIZE_THRESHOLD = 50

In [3]:
DIRECTORY = "brady129"
DISTANCE = 30
MIN_NEIGHBORS = 0

print("Loading skeletons, creating graphs")
vertices, edges, vertex_properties, G = load_skels_from_dir(DIRECTORY)

verts_and_idx = np.hstack([vertices, np.arange(len(vertices)).reshape(-1, 1)])

subgraphs = list(sorted(nx.connected_components(G), key=len, reverse=True))
nx_subgraphs = [G.subgraph(subgraph) for subgraph in subgraphs]
sG = nx_subgraphs[0]

# root_edges = np.array(sG.edges)

# root_indices = np.unique(root_edges.copy().flatten())
root_indices = sG.nodes
root_verts = vertices[root_indices, :]
root_degrees = np.array(G.degree(root_indices))
root_leaves = root_degrees[root_degrees[:, 1] == 1, 0]
leaf_verts = vertices[root_leaves, :]
root_mean = np.mean(root_verts, axis=0).reshape((1,3))


Loading skeletons, creating graphs
Loading skeleton data from pickle


In [4]:
subgraph_vectors = [get_subgraph_vector(subgraph, vertices, root_mean) for subgraph in nx_subgraphs[1:]]

In [7]:
mean_root_vert = np.average(root_verts, axis=0).reshape((1,3))


mlab.clf()
sg1 = G.subgraph(subgraphs[4200])
sg2 = G.subgraph(subgraphs[4201])
plot_subgraph(sg1, vertices)
plot_subgraph(sg2, vertices)
s_vector, distal_leaf = get_subgraph_vector(sg1, vertices, mean_root_vert, get_distal_leaf=True)
d_vector = get_subgraph_vector(sg2, vertices, mean_root_vert)

s_verts = vertices[sg1.nodes, :]
d_verts = vertices[sg2.nodes, :]
distances = get_elliptical_distance(s_verts, d_verts, 0.8, s_vector, 0.8, d_vector)
print(spatial.distance.cdist(s_verts, d_verts))
print(distances)
mlab.show()
#get_elliptical_distance(s_verts, d_verts, alpha, vector)

[[79.22114648 79.22528959 79.22743174 79.222255   79.22098694]
 [79.99784683 80.00078414 80.002567   79.99846171 79.99782762]
 [79.3186228  79.3224113  79.32444932 79.31958516 79.31850508]
 [79.61538218 79.61857122 79.62043165 79.61609846 79.61533459]
 [80.18116284 80.18408577 80.18586232 80.18177314 80.1811446 ]]
[[36.84562368 36.96521667 37.00026231 36.89496502 36.83149496]
 [37.16848043 37.28771541 37.32265482 37.21767492 37.15439353]
 [36.8813744  37.00085524 37.03586813 36.93066939 36.86725899]
 [37.00145722 37.12075533 37.15571437 37.05067702 36.98736328]
 [37.25105964 37.37029953 37.40523968 37.30025661 37.23697191]]


In [ ]:
np.repeat()

In [ ]:
pickle_name = "distance_matrix_{}.pickle".format(DIRECTORY)
if os.path.exists(pickle_name):
    print("Reading distance matrix from file: {}".format(pickle_name))
    with open(pickle_name, "rb") as fi:
        distance_matrix, mst = pickle.load(fi)
else:
    print("Calculating distance matrix:")
    non_root_indices = np.delete(np.arange(vertices.shape[0]), root_indices)
    nrG = G.subgraph(item for subgraph in subgraphs[1:] for item in subgraph)

    print("Finding connected component leaves")
    nr_degrees = np.array(nrG.degree())
    nr_leaf_indices = nr_degrees[nr_degrees[:, 1] == 1, 0]
    # nr_leaf_vertices = vertices[nr_leaf_indices, :]

    indices_by_subgraph = np.array([[item, idx] for idx, subgraph in enumerate(subgraphs) for item in subgraph])
    end_indices = np.concatenate([nr_leaf_indices, root_indices])
    end_indices_sub = indices_by_subgraph[np.isin(indices_by_subgraph, end_indices)[:, 0], :]

    distance_matrix = sparse.lil_matrix((vertices.shape[0], vertices.shape[0]))

    distance_matrix[edges[:, 0], edges[:, 1]] = 10 ^ -6

    ct = 0

    print("Finding neighbors")
    # TODO: Find the optimal path (according to the cost function) between all pairwise subgraphs, then optimize that
    for leaf_idx, subgraph in end_indices_sub[end_indices_sub[:, 1] != 0, :]:
        leaf = vertices[leaf_idx, :]
        _end_indices = end_indices_sub[end_indices_sub[:, 1] != subgraph, 0]
        end_verts = verts_and_idx[_end_indices, :]
        max_distance = DISTANCE
        neighbors = np.zeros((0, 0))
        neighbors = get_adjacent_points(leaf, end_verts, max_distance)
        while len(neighbors) < MIN_NEIGHBORS:
            neighbors = get_adjacent_points(leaf, end_verts, max_distance)
            #         print("Got {} neighbors, max_distance={}".format(len(neighbors), max_distance))
            max_distance = max_distance + 10
        leaf = leaf.reshape(1, -1)
        # TODO: use elliptical distance here
        # distances = spatial.distance_matrix(leaf, neighbors[:, :3])
        #     print(distances)
        sg_vector = get_subgraph_vector(G.subgraph(subgraphs[subgraph]), vertices, mean_root_vert=root_mean)
#         print(sg_vector.shape)
        distances = get_elliptical_distance(leaf, neighbors[:, :3], alpha=0.8, vector=sg_vector)
        distance_matrix[leaf_idx, neighbors[:, 3].astype(int)] = distances
        ct = ct + 1
        
    print("Calculating MST")
    weighted_G = nx.from_scipy_sparse_matrix(distance_matrix)
    mst = nx.minimum_spanning_tree(weighted_G)

    print("Writing distance matrix and MST: {}".format(pickle_name))
    with open(pickle_name, "wb") as fi:
        pickle.dump([distance_matrix, mst], fi)

In [ ]:
mst_subgraphs = list(sorted(nx.connected_components(mst), key=len, reverse=True))
root_mst = mst.subgraph(mst_subgraphs[0])
mst_edges = np.array(root_mst.edges)

print("Plotting")
mlab.clf()
plot_edges(mst_edges, vertices)


root_edges = np.array(sG.edges)

plot_edges(root_edges, vertices, color=(0,0,0), line_width=3.0)


# plot_edges(mst_edges, vertices, line_width=4.0)
# plot_edges(edges, vertices)
# mlab.points3d(leaf_verts[:, 0], leaf_verts[:, 1], leaf_verts[:, 2], color=(1,0,0), scale_factor=2)

mlab.show()

In [ ]:
vertices[1688]

In [ ]:
vertices[116252]

In [ ]:
end_indices_sub[end_indices_sub[:, 0]==1688, :]

In [ ]:
subgraphs[4728]

In [ ]:
subgraph = G.subgraph(subgraphs[4728])
mean_root_vert = root_mean

degrees = np.array(subgraph.degree())
leaf_indices = degrees[degrees[:, 1] == 1, 0]
leaf_verts = vertices[leaf_indices, :]
distal_leaf = leaf_verts[np.argmax(spatial.distance.cdist(mean_root_vert, leaf_verts)), :]
root_vector = np.mean(vertices[subgraph.nodes, :] - distal_leaf, axis=0)
norm = np.linalg.norm(root_vector)
if norm != 0:
    root_vector = root_vector/norm


root_vector

In [ ]:
mlab.points3d(s_verts[:,0], s_verts[:,1], s_verts[:,2])
plot_edges(root_edges, vertices, color=(0,0,0), line_width=3.0)

In [ ]:
a = np.arange(12).reshape((4,3))
b = np.arange(2,8).reshape((2,3))
print(a)
print(b)

In [ ]:
b_br = b[:, None, None, :]
res = b_br-a
print(res)
dx = res[:, :, :, 0]
dy = res[:, :, :, 1]
dz = res[:, :, :, 2]

sq_distance = dx*dx + dy*dy + dz*dz
print(sq_distance)
alpha = 0.9
final_distance = sq_distance - alpha * np.square((dx*vector[:, 0]+dy*vector[:, 1]+dz*vector[:, 2]))
final_distance = np.transpose(final_distance.reshape(b.shape[0], a.shape[0]))
print(final_distance)
print(final_distance.shape)
# res_sum = np.sum(res, axis=3)
# final_distance = res_sum.reshape((b.shape[0], a.shape[0]))
# a: m x n
# b: o x n
# f(a,b): o x 1 x m x n
# print(res.reshape(2,2,3))
# np.sum(, axis=0)

In [ ]:
def get_subgraph_vector_1(subgraph, vertices, mean_root_vert):
    degrees = np.array(subgraph.degree())
    leaf_indices = degrees[degrees[:, 1] == 1, 0]
    leaf_verts = vertices[leaf_indices, :]
    distal_leaf = leaf_verts[np.argmax(spatial.distance.cdist(mean_root_vert, leaf_verts)), :]
    root_vector = np.mean(vertices[subgraph.nodes, :] - distal_leaf, axis=0)
    root_vector = root_vector/np.linalg.norm(root_vector)
    return root_vector.reshape([1, -1]), distal_leaf.reshape([1, -1])

mlab.clf()

subgraph = G.subgraph(subgraphs[4200])
subgraph_edges = np.array(subgraph.edges)
plot_subgraph(subgraph, vertices)

root_vector, distal_leaf = get_subgraph_vector_1(subgraph, vertices, mean_root_vert)

mlab.quiver3d(distal_leaf[:, 0], distal_leaf[:, 1], distal_leaf[:, 2], vector[:, 0],
                     vector[:, 1],
                     vector[:, 2], scalars=np.array([20]))
mlab.show()

In [ ]:
np.linalg.norm(res[0,0,:,:], axis=1)

In [ ]:
distal_leaf[:, 1]

In [ ]:
os.unlink(pickle_name)

In [ ]:
np.dot(u, v)/np.linalg.norm(u)*np.linalg.norm(v)